<a href="https://colab.research.google.com/github/asad-bukhari/student_report_system/blob/main/student_report_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re  # Import the regular expression module


def get_student_info() -> str:
    """
    Prompts the user to enter student information (name, age, marks)
    and returns the information as a formatted string.
    Includes robust input validation using regular expressions.

    Returns:
        str: A string containing the student's name, age, and marks,
             separated by commas, and ending with a newline character.
             Returns an empty string if no information is entered.
    """
    while True:
        name = input("Enter Student's Name: ").strip()
        if not name:
            print("❌ Name cannot be empty")
            continue  # Go back to the beginning of the while loop

        # Use regular expression for age validation (ensure it's a number >= 5)
        age = input("Enter Student's Age: ").strip()
        if not re.match(r"^\d+$", age) or int(age) < 5:
            print("❌ Invalid age. Must be a number >= 5.")
            continue

        # Use regular expression for marks validation (ensure it's a number between 0 and 100)
        marks = input("Enter Student's Marks: ").strip()
        if not re.match(r"^\d+$", marks) or not (0 <= int(marks) <= 100):
            print("❌ Invalid marks. Must be a number between 0 and 100.")
            continue

        return f"{name},{age},{marks}\n"


def save_student(line: str, file_name: str = "students.txt") -> None:
    """
    Appends a student's information to a file.  Handles potential errors.

    Args:
        line (str): The student's information as a comma-separated string
                      (e.g., "John Doe,20,85\n").
        file_name (str, optional): The name of the file to save to.
                      Defaults to "students.txt".
    Returns:
        None
    """
    try:
        with open(file_name, "a") as file:
            file.write(line)
    except (IOError, OSError) as e:
        print(f"❌ Error writing to file '{file_name}': {e}")
        # Consider re-raising the exception or returning a specific error code/message
        # raise  # Option 1: Re-raise the exception
        # return 1 #option 2: return error code


def read_students(file_name: str = "students.txt") -> list[str]:
    """
    Reads all student information from a file.  Handles file not found and other errors.

    Args:
        file_name (str, optional): The name of the file to read from.
            Defaults to "students.txt".

    Returns:
        list[str]: A list of strings, where each string represents
                     a line from the file (i.e., one student's record).
                     Returns an empty list if the file does not exist or is empty,
                     or if an error occurs.
    """
    try:
        with open(file_name, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"❌ File '{file_name}' not found. Returning empty list.")
        return []  # Return an empty list if the file doesn't exist
    except (IOError, OSError) as e:
        print(f"❌ Error reading from file '{file_name}': {e}")
        return []  # Return empty list in case of other errors.  Consider re-raising.
        # raise  #option to re-raise


def process_student_line(line: str) -> None:
    """
    Processes a single line of student data, extracts the information,
    and prints the student's name, age, marks, and result.  Handles potential errors
    during data conversion.

    Args:
        line (str): A string containing the student's information
                      in comma-separated format (e.g., "John Doe,20,85\n").
    Returns:
       None
    """
    try:
        name, age, marks_str = line.strip().split(",")
        age = int(age)
        marks = int(marks_str)  # Convert marks to integer *before* further use
    except ValueError:
        print(f"❌ Error: Invalid data format in line: '{line.strip()}'. Skipping.")
        return  # Skip processing this line if there's a format error

    print(f"Name: {name}")
    print(f"Age: {age}")
    print(f"Marks: {marks}")

    if marks >= 90:
        print("Result: Excellent")
    elif marks >= 75:
        print("Result: Good")
    elif marks >= 50:
        print("Result: Pass")
    else:
        print("Result: Fail")
    print("-" * 40)  # Print a separator line


def search_student_by_name() -> None:
    """
    Searches for a student by name in the 'students.txt' file.
    Prompts the user for the name to search.  If a match is found,
    the student's information is displayed using process_student_line().
    Handles the case where the file is empty.
    """
    name_to_find = input("Enter name to search: ").strip().lower()
    students = read_students()

    if not students:
        print("❌ No student records found to search.")
        return

    found = False
    for line in students:
        name, age, marks = line.strip().split(",")
        if name.strip().lower() == name_to_find:
            print("\n🎯 Student Found:")
            process_student_line(line)
            found = True
            break  # Exit the loop after finding the student

    if not found:
        print("❌ Student not found.")



def delete_student_by_name() -> None:
    """
    Deletes a student's record from the 'students.txt' file based on their name.
    Prompts the user for the name of the student to delete.  Handles edge cases
    like empty file and file errors.
    """
    name_to_delete = input("Enter the name of the student to delete: ").strip().lower()
    students = read_students()

    if not students:
        print("❌ No student records found to delete.")
        return

    updated_students = []
    found = False
    for line in students:
        name, age, marks = line.strip().split(",")
        if name.strip().lower() != name_to_delete:
            updated_students.append(line)  # Keep this student's record
        else:
            found = True  # Student found, but not added to updated_students

    if found:
        try:
            with open("students.txt", "w") as file:
                file.writelines(updated_students)  # Overwrite the file
            print(f"✅ Student '{name_to_delete}' has been deleted.")
        except (IOError, OSError) as e:
            print(f"❌ Error writing to file: {e}")
            print("❌ Deletion failed.") # Inform the user that deletion failed.
    else:
        print("❌ Student not found.")



def update_student_by_name() -> None:
    """
    Updates the age and/or marks of a student in the 'students.txt' file based on their name.
    Prompts the user for the name of the student to update, and then prompts for new age and marks.
    Handles cases where the file is empty or the student is not found.
    """
    name_to_update = input("Enter the name of the student to update: ").strip().lower()
    students = read_students()

    if not students:
        print("❌ No student records found to update.")
        return

    updated_students = []
    found = False

    for line in students:
        name, age, marks = line.strip().split(",")
        if name.strip().lower() == name_to_update:
            found = True
            print(f"Current age: {age}, Current marks: {marks}")
            new_age = input("Enter new age (or press Enter to keep same): ").strip()
            new_marks = input("Enter new marks (or press Enter to keep same): ").strip()

            if new_age:
                if re.match(r"^\d+$", new_age) and int(new_age) >= 5:
                    age = new_age
                else:
                    print("❌ Invalid age. Keeping old value.")

            if new_marks:
                if re.match(r"^\d+$", new_marks) and 0 <= int(new_marks) <= 100:
                    marks = new_marks
                else:
                    print("❌ Invalid marks. Keeping old value.")

            updated_line = f"{name},{age},{marks}\n"
            updated_students.append(updated_line)
        else:
            updated_students.append(line)

    if found:
        try:
            with open("students.txt", "w") as file:
                file.writelines(updated_students)
            print(f"✅ Student '{name_to_update}' updated successfully.")
        except (IOError, OSError) as e:
            print(f"❌ Error writing to file: {e}")
            print("❌ Update failed")
    else:
        print("❌ Student not found.")



def main_menu() -> None:
    """
    Displays the main menu and handles user interaction.
    This is the entry point of the program.  Implements a loop
    to keep the menu running.
    """
    while True:
        print("\n📋 Student Report System")
        print("1. Add Student")
        print("2. View All Students")
        print("3. Exit")
        print("4. Search Student by Name")
        print("5. Delete Student by Name")
        print("6. Update Student by Name")

        choice = input("Enter your choice (1/2/3/4/5/6): ").strip()

        if choice == "1":
            student_info = get_student_info()
            save_student(student_info)
            print("✅ Student saved successfully!")
        elif choice == "2":
            print("\n📄 All Student Records")
            students = read_students()
            if not students:
                print("No student records found.")
            else:
                for line in students:
                    process_student_line(line)
        elif choice == "3":
            print("👋 Exiting the program. Goodbye!")
            break
        elif choice == "4":
            search_student_by_name()
        elif choice == "5":
            delete_student_by_name()
        elif choice == "6":
            update_student_by_name()
        else:
            print("❌ Invalid choice. Please select a number from the menu.")



if __name__ == "__main__":
    main_menu()  # Call the main function to start the program



📋 Student Report System
1. Add Student
2. View All Students
3. Exit
4. Search Student by Name
5. Delete Student by Name
6. Update Student by Name
Enter your choice (1/2/3/4/5/6): 2

📄 All Student Records
❌ File 'students.txt' not found. Returning empty list.
No student records found.

📋 Student Report System
1. Add Student
2. View All Students
3. Exit
4. Search Student by Name
5. Delete Student by Name
6. Update Student by Name
Enter your choice (1/2/3/4/5/6): 1
Enter Student's Name: asad
Enter Student's Age: 28
Enter Student's Marks: 99
✅ Student saved successfully!

📋 Student Report System
1. Add Student
2. View All Students
3. Exit
4. Search Student by Name
5. Delete Student by Name
6. Update Student by Name
Enter your choice (1/2/3/4/5/6): 2

📄 All Student Records
Name: asad
Age: 28
Marks: 99
Result: Excellent
----------------------------------------

📋 Student Report System
1. Add Student
2. View All Students
3. Exit
4. Search Student by Name
5. Delete Student by Name
6. Update S